In [ ]:
import os
import time

import numpy as np
import pandas as pd

from ipywidgets import interact

import larch

from matplotlib import pyplot as plt

os.chdir('..')
os.chdir('./catxas')
import catxas

# 1. Define where Data is Stored

In [ ]:
### Modify paths inside the '' ###

# Folder containing ONLY CXAS spectra
data_dir = r'C:\Users\ashoff\OneDrive - SLAC National Accelerator Laboratory\Co-ACCESS Shared Folder\Python Scripts and GUIs\Example Data 1 - SnO2 Reduction\Raw Data'

# Mass Spec File:
MS_filename = r'C:\Users\ashoff\OneDrive - SLAC National Accelerator Laboratory\Co-ACCESS Shared Folder\Python Scripts and GUIs\Example Data 1 - SnO2 Reduction\20210614_Sievers_SnO2_H2View1.csv'

# AmP LabView File:
LV_filename = r'C:\Users\ashoff\OneDrive - SLAC National Accelerator Laboratory\Co-ACCESS Shared Folder\Python Scripts and GUIs\Example Data 1 - SnO2 Reduction\20210614_Sievers_SnO2_H2.txt'

# 2. Define BL Data Structure

In [ ]:
### Define the layout of the xas data file and how spectra are calcualted
cxas_data_structure = {
    'time stamp': True,
    'time on line': 5,
    'time format': '# This Scan Create Date:\t%m/%d/%Y %I:%M:%S %p',
    'padded scan numbers': True,
    'column names': ['Encoder', 'Energy', 'ADC_01', 'ADC_02', 'ADC_03', 'ADC_04', 'ADC_05', 'ADC_06', 'ADC_07', 'ADC_08'],
    'energy column' : 'Energy', # Energy [eV]
    'sample numerator': 'ADC_01', # I0
    'sample denominator': 'ADC_02', # I1
    'sample ln': True,
    'sample invert': False,
    'reference numerator': 'ADC_02', # I1
    'reference denominator': 'ADC_03', # I2
    'reference ln': True,
    'reference invert': False   
}

#  3. Extract Timestamps of BL data

In [ ]:
time_stamp = cxas_data_structure['time stamp']
time_line = cxas_data_structure['time on line']
time_format = cxas_data_structure['time format']
padded = cxas_data_structure['padded scan numbers']


CXAS_summary = catxas.CXAS_Sorted(data_dir,
                   time_stamp=time_stamp, time_line=time_line, time_format=time_format,
                   padded=padded)

In [ ]:
CXAS_summary

# 4. Import MS and LV data

In [ ]:
MS_data = catxas.ReadMSData(MS_filename)

LV_data = catxas.ReadLVData(LV_filename)

# 5. Interpolate MS and LV data onto CXAS timestamps

In [ ]:
MS_interp = catxas.mergeindex(CXAS_summary, MS_data) 
LV_interp = catxas.mergeindex(CXAS_summary, LV_data)

# 6. Concatenate BL Filenames with MS and LV data

In [ ]:
process_params = pd.concat([CXAS_summary['File Name'],MS_interp, LV_interp], axis = 1)
process_params

# 7. Save Dataframe to CSV file

In [ ]:
### Modify paths inside the '' ###

# Folder where to save file
result_dir = r'C:\Users\ashoff\OneDrive - SLAC National Accelerator Laboratory\Co-ACCESS Shared Folder\Python Scripts and GUIs\Example Data 1 - SnO2 Reduction\Results'

# Name of file
result_fname = 'SnO2_Spectra_Process_Correlation'

# Header formats
index_label = 'Clock Time' #None

In [ ]:
process_params.to_csv(os.path.join(result_dir, result_fname + '.csv'), sep=',', 
                      na_rep='', header=True, index=True, index_label=index_label)